<a href="https://colab.research.google.com/github/Riky2014/NAPDE/blob/main/Project1/Cp3/tentativo_disperato_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cvxpy as cvx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import make_axes_locatable

!pip -q install pyDOE
from pyDOE import lhs

tf.random.set_seed(42)
np.random.seed(42)

  Preparing metadata (setup.py) ... done


In [2]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp3

Cloning into 'ScientificTools'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 672 (delta 34), reused 1 (delta 1), pack-reused 601
Receiving objects: 100% (672/672), 140.57 MiB | 25.80 MiB/s, done.
Resolving deltas: 100% (232/232), done.
/content/ScientificTools/Project1/Cp3


In [3]:
CP3data = np.load("CP3data.npz")
CP3data = CP3data['arr_0']

CP3estimate = np.load("CP3field.npz")
CP3estimate = CP3estimate['arr_0']

In [4]:
mesh_size = 151**2
num_samples = 100
A = np.zeros((mesh_size, num_samples))

for i in range(num_samples):
  A[:, i] = CP3estimate[i][0].flatten()

[U,s,V] = np.linalg.svd(A, full_matrices=False)

k = 10
p = 100
perm = np.random.choice(mesh_size, size=p, replace=False)

C = np.zeros((p,mesh_size))
for i in range(p):
    C[i,perm[i]] = 1.0

Theta = C @ U
coeff_mat = np.zeros((k, num_samples))

for i in range (num_samples):
  u = A[:,i]
  y = C @ u
  s_c = cvx.Variable(U.shape[1])
  constraints = [Theta @ s_c == y]
  obj = cvx.Minimize(cvx.norm(s_c, 1))
  prob = cvx.Problem(obj, constraints)
  prob.solve()
  coeff_mat[:, i] = s_c.value[:k]

c1_min, c1_max = np.min(coeff_mat[0, :]), np.max(coeff_mat[0, :])
c2_min, c2_max = np.min(coeff_mat[1, :]), np.max(coeff_mat[1, :])
c3_min, c3_max = np.min(coeff_mat[2, :]), np.max(coeff_mat[2, :])
c4_min, c4_max = np.min(coeff_mat[3, :]), np.max(coeff_mat[3, :])
c5_min, c5_max = np.min(coeff_mat[4, :]), np.max(coeff_mat[4, :])
c6_min, c6_max = np.min(coeff_mat[5, :]), np.max(coeff_mat[5, :])
c7_min, c7_max = np.min(coeff_mat[6, :]), np.max(coeff_mat[6, :])
c8_min, c8_max = np.min(coeff_mat[7, :]), np.max(coeff_mat[7, :])
c9_min, c9_max = np.min(coeff_mat[8, :]), np.max(coeff_mat[8, :])
c10_min, c10_max = np.min(coeff_mat[9, :]), np.max(coeff_mat[9, :])

delta_c1 = c1_max - c1_min
delta_c2 = c2_max - c2_min
delta_c3 = c3_max - c3_min
delta_c4 = c4_max - c4_min
delta_c5 = c5_max - c5_min
delta_c6 = c6_max - c6_min
delta_c7 = c7_max - c7_min
delta_c8 = c8_max - c8_min
delta_c9 = c9_max - c9_min
delta_c10 = c10_max - c10_min

print(f"c1 in ({c1_min}, {c1_max})")
print(f"c2 in ({c2_min}, {c2_max})")
print(f"c3 in ({c3_min}, {c3_max})")
print(f"c4 in ({c4_min}, {c4_max})")
print(f"c5 in ({c5_min}, {c5_max})")
print(f"c6 in ({c6_min}, {c6_max})")
print(f"c7 in ({c7_min}, {c7_max})")
print(f"c8 in ({c8_min}, {c8_max})")
print(f"c9 in ({c9_min}, {c9_max})")
print(f"c10 in ({c10_min}, {c10_max})")

basis = U[:, :k]

c1 in (-14901.762094931402, -12144.026767259482)
c2 in (-1643.3543282356468, 1582.2151985821024)
c3 in (-1666.6222565459314, 1707.616360833575)
c4 in (-776.3212422610503, 1280.585831484112)
c5 in (-1093.9752515232494, 1047.6008504668707)
c6 in (-717.0424992233817, 537.6185968969928)
c7 in (-469.9943812071792, 438.8180282175097)
c8 in (-328.7863116122087, 528.7552626498631)
c9 in (-257.8055527194194, 317.60595986928547)
c10 in (-216.0399309556303, 357.6237000279386)


In [12]:
# collocation points
Ncl = 151**2
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

In [5]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))

# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,coeff,PINN):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = PINN(input_data)
    r_pred   = r_PINN(xcl,ycl,coeff_1,coeff_2,coeff_3,coeff_other)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.abs(r_pred))

    # bc
    y0 = tf.constant([-0.517409965],dtype=tf.float64)
    mse_bc= tf.pow( PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), y0] ) ) ) ,2)

    #penalty
    mse_penalty = penalty(coeff_other[0],0,1)+penalty(coeff_other[1],0,1)+penalty(coeff_other[2],0,1)+penalty(coeff_other[3],0,1)+penalty(coeff_other[4],0,1)+penalty(coeff_other[5],0,1)+penalty(coeff_other[6],0,1)+penalty(coeff_1,0,1)+penalty(coeff_2,0,1)+penalty(coeff_3,0,1)

    return mse_meas + mse_r + mse_bc + mse_penalty

def loss2(xcl,ycl,xmeas,ymeas,umeas,PINN):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = PINN(input_data)
    r_pred   = r_PINN2(xcl,ycl,PINN)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.abs(r_pred))

    # bc
    y0 = tf.constant([-0.517409965],dtype=tf.float64)
    mse_bc= tf.pow(PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), y0] ))  ) ,2)

    return mse_meas + mse_r + mse_bc


@tf.function
def r_PINN(x,y,coeff,PINN):
    input_data=tf.concat([x,y],1)
    u = PINN(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    pi = tf.constant(np.pi,dtype=tf.float64)
    theta_fiber = tf.constant([0.13757033666666668] ,dtype=tf.float64)
    a_ratio = tf.constant([5.896298503333333], dtype=tf.float64)
    theta0 = pi/2 - theta_fiber

    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D_00 = 1 / a_ratio * a[0]**2 + b[0]**2
    D_01 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / a_ratio * a[1]**2 + b[1]**2

    c1 = c1_min + delta_c1 * coeff_1
    c2 = c2_min + delta_c2 * coeff_2
    c3 = c3_min + delta_c3 * coeff_3
    c4 = c4_min + delta_c4 * coeff_other[0]
    c5 = c5_min + delta_c5 * coeff_other[1]
    c6 = c6_min + delta_c6 * coeff_other[2]
    c7 = c7_min + delta_c7 * coeff_other[3]
    c8 = c8_min + delta_c8 * coeff_other[4]
    c9 = c9_min + delta_c9 * coeff_other[5]
    c10 = c10_min + delta_c10 * coeff_other[6]

    coeff_true = tf.expand_dims(tf.concat([c1[0],c2[0],c3[0],c4,c5,c6,c7,c8,c9,c10], 0), 1)

    return   (((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y)))  - (1/(basis@coeff_true))**2

@tf.function
def r_PINN2(x,y,PINN):
    input_data=tf.concat([x,y],1)
    u = PINN(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    pi = tf.constant(np.pi,dtype=tf.float64)
    theta_fiber = tf.constant([0.13757033666666668] ,dtype=tf.float64)
    a_ratio = tf.constant([5.896298503333333], dtype=tf.float64)
    theta0 = pi/2 - theta_fiber

    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D_00 = 1 / a_ratio * a[0]**2 + b[0]**2
    D_01 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / a_ratio * a[1]**2 + b[1]**2

    return  (((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y)))  - (1/100)**2


# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,coeff):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,coeff,model)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grads_weight= tape.gradient(loss_value,coeff)
    return loss_value, grads, grads_weight
    # neural network weight gradients

@tf.function
def grad2(model,xcl,ycl,xmeas,ymeas,umeas):
    with tf.GradientTape(persistent=True) as tape:
        loss_value2 = loss2(xcl,ycl,xmeas,ymeas,umeas,model)
        grads2 = tape.gradient(loss_value2,model.trainable_variables)
    return loss_value2, grads2

In [7]:
from tensorflow.keras import regularizers

def build_pinn(regularization_strength):
    PINN = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(2,),
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(64, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(128, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(64, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(32, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(1, activation=None,
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64)
    ])
    return PINN



In [8]:
def checkpoint2_solution( x, y , t, regularization_strength):

  PINN = build_pinn(regularization_strength)

  xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(x, y, t, test_size=0.05)
  xmeas_train = tf.constant(xmeas_train.reshape(19, 1), dtype=tf.float64)
  ymeas_train = tf.constant(ymeas_train.reshape(19, 1), dtype=tf.float64)
  tmeas_train = tf.constant(tmeas_train.reshape(19, 1), dtype=tf.float64)
  xmeas_val = tf.constant(xmeas_val.reshape(1, 1), dtype=tf.float64)
  ymeas_val = tf.constant(ymeas_val.reshape(1, 1), dtype=tf.float64)
  tmeas_val = tf.constant(tmeas_val.reshape(1, 1), dtype=tf.float64)

  X, Y = np.meshgrid(np.linspace(-1.5,1.5,151), np.linspace(-1.5,1.5,151))

  coeff_1 = tf.Variable([[0.5]], trainable=True, dtype=tf.float64)
  coeff_2 = tf.Variable([[0.5]], trainable=True, dtype=tf.float64)
  coeff_3 = tf.Variable([[0.5]], trainable=True, dtype=tf.float64)
  coeff_other = tf.Variable([[0.5],[0.5],[0.5],[0.5],[0.5],[0.5],[0.5]], trainable=True, dtype=tf.float64)

  tf_optimizer_PINN = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  tf_optimizer_PINN.build(PINN.trainable_variables)
  tf_optimizer_c1 = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  tf_optimizer_c1.build([coeff_1])
  tf_optimizer_c2 = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  tf_optimizer_c2.build([coeff_2])
  tf_optimizer_c3 = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  tf_optimizer_c3.build([coeff_3])
  tf_optimizer_other = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  tf_optimizer_other.build([coeff_other])


  print()

  for iter in range(100):

    loss_value2,grads2 = grad2(PINN, xcl,ycl,xmeas_train, ymeas_train, tmeas_train)

    tf_optimizer_PINN.apply_gradients(zip(grads2,PINN.trainable_variables))

    loss_value_val2, _= grad2(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value2)
      tf.print('loss_val_param =' , loss_value_val2)

      c1 = c1_min + delta_c1 * coeff_1
      c2 = c2_min + delta_c2 * coeff_2
      c3 = c3_min + delta_c3 * coeff_3
      c4 = c4_min + delta_c4 * coeff_other[0]
      c5 = c5_min + delta_c5 * coeff_other[1]
      c6 = c6_min + delta_c6 * coeff_other[2]
      c7 = c7_min + delta_c7 * coeff_other[3]
      c8 = c8_min + delta_c8 * coeff_other[4]
      c9 = c9_min + delta_c9 * coeff_other[5]
      c10 = c10_min + delta_c10 * coeff_other[6]

      coeff_true = tf.expand_dims(tf.concat([c1[0],c2[0],c3[0],c4,c5,c6,c7,c8,c9,c10], 0), 1)
      print(coeff_true.numpy())
      print()


  for iter in range(100):

    loss_value,grads,grads_coeff = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train,coeff)

    tf_optimizer_PINN.apply_gradients(zip(grads ,PINN.trainable_variables))
    tf_optimizer_weights.apply_gradients(zip([grads_coeff], [coeff]))

    loss_value_val, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,coeff)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      c1 = c1_min + delta_c1 * coeff_1
      c2 = c2_min + delta_c2 * coeff_2
      c3 = c3_min + delta_c3 * coeff_3
      c4 = c4_min + delta_c4 * coeff_other[0]
      c5 = c5_min + delta_c5 * coeff_other[1]
      c6 = c6_min + delta_c6 * coeff_other[2]
      c7 = c7_min + delta_c7 * coeff_other[3]
      c8 = c8_min + delta_c8 * coeff_other[4]
      c9 = c9_min + delta_c9 * coeff_other[5]
      c10 = c10_min + delta_c10 * coeff_other[6]

      coeff_true = tf.expand_dims(tf.concat([c1[0],c2[0],c3[0],c4,c5,c6,c7,c8,c9,c10], 0), 1)
      print(coeff_true.numpy())
      print()

  return regularization_strength, loss_value_val

In [9]:
num_random_search = 10
regularization_strengths = np.logspace(-6, 0, num=num_random_search)
regularization_strengths

array([1.00000000e-06, 4.64158883e-06, 2.15443469e-05, 1.00000000e-04,
       4.64158883e-04, 2.15443469e-03, 1.00000000e-02, 4.64158883e-02,
       2.15443469e-01, 1.00000000e+00])

In [13]:
ind_disp = 64
x_meas = CP3data[ind_disp][0]
y_meas = CP3data[ind_disp][1]
t_meas = CP3data[ind_disp][2]
speed_field = CP3estimate[ind_disp][0]

num_random_search = 10
regularization_strengths = np.logspace(-6, 0, num=num_random_search)

best_val_loss = float('inf')
best_regularization_strength = None


for regularization_strength in regularization_strengths:
  regularization_strength, loss_value_val= checkpoint2_solution(x_meas, y_meas, t_meas,regularization_strength)
  if  loss_value_val < best_val_loss:
      best_val_loss =  loss_value_val
      best_regularization_strength = regularization_strength
  print(f'Best Regularization strength: {best_regularization_strength}, Best Validation Loss: {best_val_loss}')


iter =  100
loss = [[0.00037162011846411631]]
loss_val_param = [[0.00059736754708933479]]
[[-13522.8944311 ]
 [   -30.56956483]
 [    20.49705214]
 [   252.13229461]
 [   -23.18720053]
 [   -89.71195116]
 [   -15.58817649]
 [    99.98447552]
 [    29.90020357]
 [    70.79188454]]

iter =  100
loss = [[0.00011093410504335435]]
loss_val_param = [[0.00016792537889083063]]
[[-1.48126712e+04]
 [ 6.99162361e+01]
 [-7.27507446e+01]
 [-7.48053691e+02]
 [-3.27987734e+00]
 [ 1.43119631e+02]
 [ 7.73242639e+01]
 [ 7.51417565e+01]
 [ 7.59406411e+01]
 [-2.11335994e+01]]

Best Regularization strength: 1e-06, Best Validation Loss: [[0.00016793]]

iter =  100
loss = [[0.00090673905277941142]]
loss_val_param = [[0.0018929132042861648]]
[[-13522.8944311 ]
 [   -30.56956483]
 [    20.49705214]
 [   252.13229461]
 [   -23.18720053]
 [   -89.71195116]
 [   -15.58817649]
 [    99.98447552]
 [    29.90020357]
 [    70.79188454]]

iter =  100
loss = [[0.0005294514338008633]]
loss_val_param = [[0.00052593128540